In [11]:
import numpy as np
import open3d as o3d

view_count = 16
point_data = "blocks"

## Direct Point Cloud Normal Calculation

In [12]:
pcd = o3d.io.read_point_cloud(f"../RandomPushing/data/pcs/{point_data}/mstore.pcd")
pcd = pcd.voxel_down_sample(voxel_size=0.005)
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.025, max_nn=20)
)
pcd.orient_normals_consistent_tangent_plane(k=30)

# Visualize the point cloud with smoothed normals
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

[Open3D WARNING] Read PCD failed: unable to open file: ../RandomPushing/data/pcs/spray/mstore.pcd
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


RuntimeError: [1;31m[Open3D Error] (void open3d::geometry::PointCloud::OrientNormalsConsistentTangentPlane(size_t)) /home/runner/work/Open3D/Open3D/cpp/open3d/geometry/EstimateNormals.cpp:393: [OrientNormalsConsistentTangentPlane] No normals in the PointCloud. Call EstimateNormals() first.
[0;m

## Better Point Cloud Normal Calculation Method
- Loop through all camera views
- Rotate the views to get a better normal direction estimation (Can still be improved)
- Estimate the point normals
- Combine the point clouds with the mstore function
- Downsample the points and use a voting system for averaging the normals

### Calculate point normals for individual point clouds

In [13]:
step = -2.*np.pi/float(view_count)
pcds = []
for i in range(view_count):
    pcd = o3d.io.read_point_cloud(f"../RandomPushing/data/pcs/{point_data}/point_cloud_{i}.pcd")
    pcd = pcd.voxel_down_sample(voxel_size=.003)
    pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

    # Center point cloud
    min_coor = np.array([
        min([p[0] for p in pcd.points]),
        min([p[1] for p in pcd.points]),
        min([p[2] for p in pcd.points])
    ])

    max_coor = np.array([
        max([p[0] for p in pcd.points]),
        max([p[1] for p in pcd.points]),
        max([p[2] for p in pcd.points])
    ])

    midpoint = (max_coor+min_coor)*.5

    for j in range(len(pcd.points)):
        pcd.points[j] -= midpoint
        tmp = pcd.points[j][1]
        pcd.points[j][1] = pcd.points[j][2]
        pcd.points[j][2] = tmp

    # Rotate point cloud
    angle = i*step
    rotation_matrix = np.array([
        [np.cos(angle), 0., np.sin(angle)],
        [0., 1., 0.],
        [-np.sin(angle), 0., np.cos(angle)]
    ])
    for j in range(len(pcd.points)):
        pcd.points[j] = rotation_matrix @ pcd.points[j]

    # Calculate and display normals
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=.025, max_nn=20))
    
    pcd.orient_normals_consistent_tangent_plane(k=30)
    # o3d.visualization.draw_geometries([pcd], point_show_normal=True)

    # Rotate back (king of ugly)
    angle = -i*step
    rotation_matrix = np.array([
        [np.cos(angle), 0., np.sin(angle)],
        [0., 1., 0.],
        [-np.sin(angle), 0., np.cos(angle)]
    ])
    for j in range(len(pcd.points)):
        pcd.points[j] = rotation_matrix @ pcd.points[j]
        pcd.normals[j] = rotation_matrix @ pcd.normals[j]
    
    pcds.append(pcd)

### Combine the point clouds

In [14]:
from PointClouds.Registration import mstorePCR

# Join viewpoints
pointCloud = mstorePCR(pcds)
o3d.visualization.draw_geometries([pointCloud], point_show_normal=True)

# Adjust orientations
pointCloud.orient_normals_consistent_tangent_plane(k=30)
o3d.visualization.draw_geometries([pointCloud], point_show_normal=True)

# Downsample
pointCloud = pointCloud.voxel_down_sample(voxel_size=.005)
o3d.visualization.draw_geometries([pointCloud], point_show_normal=True)


### Alternative downsampling method combining point normals

In [15]:
from PointClouds.Downsampling import pointNormalVoxels, normalVoting

pointCloud = mstorePCR(pcds)
pointsNormals, dims = pointNormalVoxels(pointCloud)
points = []
normals = []

# Very slow (will change in the future)
for x in range(dims[0]):
    for y in range(dims[1]):
        for z in range(dims[2]):
            if len(pointsNormals[x][y][z]):
                ps = [p[0] for p in pointsNormals[x][y][z]]
                points.append(sum(ps)/len(ps))
                ns = [p[1] for p in pointsNormals[x][y][z]]
                if len(ns) > 2:
                    normals.append(normalVoting(np.array(ns)))
                else:
                    normals.append(sum(ns)/len(ns))

points = np.array(points)
normals = np.array(normals)
final_pcd = o3d.geometry.PointCloud()
final_pcd.points = o3d.utility.Vector3dVector(points)
final_pcd.normals = o3d.utility.Vector3dVector(normals)

o3d.visualization.draw_geometries([final_pcd], point_show_normal=True)